In [1]:
import os
import numpy as np

from vae import Sampling, VAE

from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

In [2]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 14, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 7, 7, 64)     18496       conv2d[0][0]                     
__________________________________________________________________________________________________
flatten (Flatten)               (None, 3136)         0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [3]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 1)         289       
Total params: 65,089
Trainable params: 65,089
Non-trainable params: 0
_______________________________________________________

In [4]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data(path=os.path.expanduser('~/dev/data/mnist/mnist.npz'))

In [5]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

In [6]:
vae = VAE(encoder, decoder)

In [7]:
vae.compile(optimizer='adam')

In [8]:
vae.fit(x_train, epochs=50, batch_size=128)

Epoch 1/50
469/469 [==============================] - 32s 68ms/step - loss: 260.4938 - reconstruction_loss: 209.9645 - kl_loss: 2.9218
Epoch 2/50
 18/469 [>.............................] - ETA: 30s - loss: 187.9295 - reconstruction_loss: 185.6503 - kl_loss: 2.6795

KeyboardInterrupt: 

In [45]:
vae.save_weights('./results/model_04/vae_weights', save_format='tf')

In [8]:
vae.load_weights('./results/model_04/vae_weights')

In [9]:
# %%timeit

z_mean, z_log_var, z = vae.encoder(x_test)

reconstruction = vae.decoder(z_mean)
reconstruction_errors = tf.reduce_mean(keras.losses.mean_squared_error(x_test, reconstruction), axis=(1, 2))

In [29]:
L = 100

re = []

for l in tqdm(range(L)):
    z = Sampling()([z_mean, z_log_var])
    reconstruction = vae.decoder(z)
    reconstruction_errors = tf.reduce_sum(keras.losses.mean_squared_error(x_test, reconstruction), axis=(1, 2))
    re.append(reconstruction_errors)
    
reconstruction_errors = tf.reduce_mean(re, 0)

In [10]:
reconstruction_errors_prioritization = np.argsort(reconstruction_errors)[::-1]

# np.save('./results/model_04/reconstruction_errors_prioritization.npy', reconstruction_errors_prioritization)